In [1]:
import pandas as pd
from talkingtomachines.management.experiment import AItoAIInterviewExperiment, AItoAIConversationalExperiment

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Load and Preprocess Demographic Data

In [2]:
demographic_col_mapping = {
    'gender':'gender', 
    'age':'age',
    'college':'college', 
    'white':'white', 
    'knowledge_counter':'knowledge_counter', 
    'extraversion':'extraversion', 
    'agreeable':'agreeable',
    'consecientious':'consecientious', 
    'emotionally_stable':'emotionally_stable', 
    'open_new_experiences':'open_new_experiences',
    'political_interest':'political_interest', 
    'strong_partisan':'strong_partisan', 
    'percent_other_party_exposure':'percent_other_party_exposure',
    'democrat':'democrat', 
    'sevenpoint_ideology':'sevenpoint_ideology', 
    'regionMidwest':'regionMidwest', 
    'regionSouth':'regionSouth',
    'regionWest':'regionWest', 
    'faminc_val':'faminc_val', 
    'republican':'republican', 
    'faminc_notsay':'faminc_notsay',
    'index':'ID'
}

demographic_cols = list(demographic_col_mapping.values())

demographics_data = pd.read_csv("reducing_political_polarisation.csv")
demographics_data = demographics_data.reset_index(drop=False).rename(columns=demographic_col_mapping)
demographics_data.head()

,ID,depolarization_index_1issue_change,affect_index_change,issue_index_global_change,other_party_change,social_distance_change,outgroup_stereotype_change,gender,age,college,white,knowledge_counter,extraversion,agreeable,consecientious,emotionally_stable,open_new_experiences,political_interest,strong_partisan,percent_other_party_exposure,democrat,sevenpoint_ideology,regionMidwest,regionSouth,regionWest,faminc_val,republican,faminc_notsay,fully_compliant,treatment_assigned,displayCondition,chatID_full,fully_compliant_CORRECT,fully_compliant_INCORRECT,fully_compliant_NONE,treatment_assigned_CORRECT,treatment_assigned_INCORRECT,treatment_assigned_NONE
0,0,0.544913,0.065741,0.008333,0.075,-0.150000,0.027778,1,50,1,1,2,5.0,2.5,1.5,3.0,2.0,4.0,0,60.0,0,4.0,True,False,False,1.525953,1,False,0,0,CONTROL,CONTROL_1,False,False,False,False,False,False
1,1,-1.050319,-0.072778,-0.070000,-0.085,0.133333,0.000000,2,51,1,1,2,6.0,3.0,2.0,3.0,2.0,4.0,0,50.0,1,3.0,True,False,False,0.588796,0,False,1,1,CORRECT,HoMHmgJylZR06D0YmVrB,True,False,False,True,False,False
2,2,0.474074,0.137778,-0.073333,0.180,-0.233333,0.000000,1,56,0,0,3,4.5,3.0,2.5,1.5,3.0,3.0,0,10.0,1,4.0,False,False,False,-0.492539,0,False,1,1,CORRECT,1UERijH7RvXUepisvbWJ,True,False,False,True,False,False
3,3,0.644359,0.067593,0.020000,-0.075,-0.166667,-0.111111,2,43,1,1,3,2.0,1.0,1.0,1.0,1.0,4.0,1,20.0,1,1.0,True,False,False,-0.636716,0,False,0,0,ROBUST,ROBUST_4,False,False,False,False,False,False
4,4,0.704299,0.045741,0.050000,0.015,-0.066667,-0.055556,1,26,1,1,3,2.5,1.0,2.5,4.5,3.0,4.0,1,5.0,1,1.0,True,False,False,-0.348361,0,False,0,0,CONTROL,CONTROL_6,False,False,False,False,False,False


In [3]:
# Split into treatment 1 and treatment 2
placebo_demographics_data = demographics_data[demographics_data["treatment_assigned"]==0].reset_index(drop=True)
treatment_demographics_data = demographics_data[demographics_data["treatment_assigned"]==1].reset_index(drop=True)

# For treatment 1, split into immigration group and gun control group


# For treatment 2, split into immigration group and gun control group



# Treatment 1 (T1) - Placebo Group on Immigration
Is there a need to implement the pure control group?
Are we allowing the subject to elaborate on their initial response?

In [4]:
model_info = "gpt-4o-mini"
experiment_context = "DiscussIt is a social mobile platform where users can have online and anonymous chat-based discussions about various topics in the United States This is one of the discussions happening on the platform."

agent_roles = {
    "Interviewer": "Assume the role of an interviewer in this discussion. Ask the subject about their thoughts about the following question: 'Do you think the benefits of immigration outweigh the potential downsides?' Remind the subject to not disclose their actual names or personal information in their response in order to allow conversations to develop freely. Once your question has been answered, you must end the discussion by saying 'Thank you for the conversation.'. Do not have the discussion by yourself as there will be another party in this discussion. Start all of your responses with 'Interviewer:'.",
    "Subject": "Assume the role of the subject in this discussion. You will be assigned a pseudonym as Jamie for this discussion. Answer any question that is posed to you by the interviewer in as much detail as possible. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not have the discussion by yourself as there will be another party in this discussion. Start all your response with 'Subject:'.",
}

treatments = {"No Treatment":""}

# Initialise AI-to-AI Conversational experiment object
placebo_immigration_group_experiment = AItoAIInterviewExperiment(
        model_info=model_info,
        experiment_context=experiment_context,
        agent_demographics=placebo_demographics_data[demographic_cols],
        agent_roles=agent_roles,
        num_agents_per_session=2,
        num_sessions=10,
        max_conversation_length=10,
        treatments=treatments,
        treatment_assignment_strategy="simple_random",
)

# print(f"Experiment ID: {placebo_immigration_group_experiment.experiment_id}")
# print()
# print(f"Session ID: {placebo_immigration_group_experiment.session_id_list}")
# print()

# print(f"Treatment Assignment:")
# for session, session_treatment in placebo_immigration_group_experiment.treatment_assignment.items():
#     print(f"Session {session}: {session_treatment}")
# print()

# print(f"Agent Assignment:")
# for session, agents in placebo_immigration_group_experiment.agent_assignment.items():
#     print(f"Session {session}:")
#     for agent in agents:
#         print(agent)
# print()

placebo_immigration_group_results = placebo_immigration_group_experiment.run_experiment(test_mode=True)

  0%|          | 0/1 [00:00<?, ?it/s]

{'system': 'DiscussIt is a social mobile platform where users can have online and anonymous chat-based discussions about various topics in the United States This is one of the discussions happening on the platform.\n\n'}



# Treatment 1 (T1) - Placebo Group on Gun Control

In [ ]:
model_info = "gpt-4o-mini"
experiment_context = "DiscussIt is a social mobile platform where users can have online and anonymous chat-based discussions about various topics in the United States. This is one of the discussions happening on the platform."

agent_roles = {
    "Interviewer": "Assume the role of an interviewer in this discussion. Ask the subject about their thoughts about the following question: 'Do you think the benefits of gun control outweigh the downsides?' Remind the subject to not disclose their actual names or personal information in their response in order to allow discussions to develop freely. Once your question has been answered, you must end the discussion by saying 'Thank you for the conversation.'. Do not have the discussion by yourself as there will be another party in this discussion. Start all of your responses with 'Interviewer:'.",
    "Subject": "Assume the role of the subject in this discussion. You will be assigned a pseudonym as Jamie for this conversadiscussiontion. Answer any question that is posed to you by the interviewer in as much detail as possible. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information aavailable to you. Do not have the discussion by yourself as there will be another party in this discussion. Start all your response with 'Subject:'.",
}

treatments = {"No Treatment":""}

# Initialise AI-to-AI Conversational experiment object
placebo_guncontrol_group_experiment = AItoAIInterviewExperiment(
        model_info=model_info,
        experiment_context=experiment_context,
        agent_demographics=placebo_demographics_data[demographic_cols],
        agent_roles=agent_roles,
        num_agents_per_session=2,
        num_sessions=10,
        max_conversation_length=10,
        treatments=treatments,
        treatment_assignment_strategy="simple_random",
)

# print(f"Experiment ID: {placebo_guncontrol_group_experiment.experiment_id}")
# print()
# print(f"Session ID: {placebo_guncontrol_group_experiment.session_id_list}")
# print()

placebo_guncontrol_group_results = placebo_guncontrol_group_experiment.run_experiment(test_mode=True)

# Treatment 2 (T2) - Treatment Group on Immigration
Is there a need to implement the chat rating part of the experiment?

In [ ]:
model_info = "gpt-4o-mini"
experiment_context = "DiscussIt is a social mobile platform where users can have online and anonymous chat-based discussions about various topics in the United States. This is one of the discussions happening on the platform between two subjects."

agent_roles = {
    "Subject 1": "Assume the role of one of the subjects in this discussion. You will be assigned a pseudonym as Jamie for this discussion. Start by sharing your thoughts about the following question with the other subject, Taylor: 'Do you think the benefits of immigration outweigh the potential downsides? Tell Taylor what you think!' In order to receive compensation from YouGov for participating in this discussion, you will have reply to messages from Taylor at least 14 times. Do not disclose your actual name or personal information in your response in order to allow conversations to develop freely. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not have the discussion by yourself as there will be another party in this discussion. Start all of your responses with 'Jamie:'.",
    
    "Subject 2": "Assume the role of one of the subjects in this discussion. You will be assigned a pseudonym as Taylor for this discussion. Start by responding to the following: 'To begin, we asked Jamie to share their stance on this question: Do you think the benefits of immigration outweigh the potential downsides? Write back to Jamie complete your first interaction on DiscussIt!' In order to receive compensation from YouGov for participating in the discussion, you will have reply to messages from Jamie at least 14 times. Do not disclose your actual name or personal information in your response in order to allow conversations to develop freely. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not have the discussion by yourself as there will be another party in this discussion. Start all of your responses with 'Taylor:'.",
}

treatments = {
    "No Party Label":"",
    "Correct Party Label":"The other party that you are conversing with is predicted to be from a different political party.",
    "Incorrect Party Label":"The other party that you are conversing with is predicted to be from the same political party",
}

# Initialise AI-to-AI Conversational experiment object
treatment_immigration_group_experiment = AItoAIConversationalExperiment(
        model_info=model_info,
        experiment_context=experiment_context,
        agent_demographics=treatment_demographics_data[demographic_cols],
        agent_roles=agent_roles,
        num_agents_per_session=2,
        num_sessions=10,
        max_conversation_length=10,
        treatments=treatments,
        treatment_assignment_strategy="simple_random",
)

# print(f"Experiment ID: {treatment_immigration_group_experiment.experiment_id}")
# print()
# print(f"Session ID: {treatment_immigration_group_experiment.session_id_list}")
# print()
# print(treatment_immigration_group_experiment.agent_assignment)

treatment_immigration_group_results = treatment_immigration_group_experiment.run_experiment(test_mode=True)

# Treatment 2 (T2) - Treatment Group on Gun Control

In [ ]:
model_info = "gpt-4o-mini"
experiment_context = "DiscussIt is a social mobile platform where users can have online and anonymous chat-based discussions about various topics in the United States. This is one of the discussions happening on the platform between two subjects."

agent_roles = {
    "Subject 1": "Assume the role of one of the subjects in this discussion. You will be assigned a pseudonym as Jamie for this discussion. Start by sharing your thoughts about the following question with the other subject, Taylor: 'Do you think the benefits of gun control outweigh the downsides? Tell Taylor what you think!' In order to receive compensation from YouGov for participating in this discussion, you will have reply to messages from Taylor at least 14 times. Do not disclose your actual name or personal information in your response in order to allow conversations to develop freely. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not have the discussion by yourself as there will be another party in this discussion. Start all of your responses with 'Jamie:'.",
    
    "Subject 2": "Assume the role of one of the subjects in this discussion. You will be assigned a pseudonym as Taylor for this discussion. Start by responding to the following: 'To begin, we asked Jamie to share their stance on this question: Do you think the benefits of gun control outweigh the downsides? Write back to Jamie complete your first interaction on DiscussIt!' In order to receive compensation from YouGov for participating in the discussion, you will have reply to messages from Jamie at least 14 times. Do not disclose your actual name or personal information in your response in order to allow conversations to develop freely. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not have the discussion by yourself as there will be another party in this discussion. Start all of your responses with 'Taylor:'.",
}

treatments = {
    "No Party Label":"",
    "Correct Party Label":"The other party that you are conversing with is predicted to be from a different political party.",
    "Incorrect Party Label":"The other party that you are conversing with is predicted to be from the same political party",
}

# Initialise AI-to-AI Conversational experiment object
treatment_guncontrol_group_experiment = AItoAIConversationalExperiment(
        model_info=model_info,
        experiment_context=experiment_context,
        agent_demographics=treatment_demographics_data[demographic_cols],
        agent_roles=agent_roles,
        num_agents_per_session=2,
        num_sessions=10,
        max_conversation_length=10,
        treatments=treatments,
        treatment_assignment_strategy="simple_random",
)

# print(f"Experiment ID: {treatment_guncontrol_group_experiment.experiment_id}")
# print()
# print(f"Session ID: {treatment_guncontrol_group_experiment.session_id_list}")
# print()

treatment_guncontrol_group_results = treatment_guncontrol_group_experiment.run_experiment(test_mode=True)